In [1]:
import requests
import time
import json
import threading
import queue
import random
import datetime
import os
import urllib3
import pickle
import re

from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from collections import defaultdict
from pprint import pprint
from config.assistant import *

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
search_words = ["컴퓨터비전", 'COMPUTER VISION',
                "영상처리", "IMAGE PROCESSING",
                "딥러닝", "DEEP LEARNING", 
                "머신러닝", "MACHINE LEARNING",
                "이미지인식", "IMAGE RECOGNITION",
                "이미지분석", "IMAGE ANALYSIS",
                "VISION AI",
                "OPENCV"
               ]

search_words = ["이미지인식"]
base_link_saram = 'https://www.saramin.co.kr'
base_link_korea = 'https://www.jobkorea.co.kr'

job_group = {}

In [3]:
def save_dictionary(job_company, company_link, job_posting, posting_link):
    global job_group
    
    # 신규 회사명 인입
    if not job_company in job_group:    
        job_group[job_company] = []
    
    for company in job_group[job_company]:
        if company['title'] == job_posting:    # 채용공고명 - 중복
            break
    else:
        job_group[job_company].append({'company': job_company, \
                                       'company_link': company_link, \
                                       'title': job_posting, \
                                       'title_link': posting_link, \
                                       'input_date': datetime.date.today().isoformat(), \
                                       'status': 'wait'
                                      })
    return None

### 잡코리아

In [6]:
# 잡코리아
# 저장된 키워드 별 검색

tic()
for search_word in search_words:

    item_count = 0
    current_page = 1
    
    # 검색 결과 리스트 페이지 별 확인
    search_loop = True
    while(search_loop):
        search_link = f'https://www.jobkorea.co.kr/Search/' \
                    +f'?stext={search_word}' \
                    +f'&tabType=recruit&Page_No={current_page}'

        response = requests.get(search_link, verify=False)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        page_items = len(soup.find_all('a', attrs={'class': 'title dev_view'}))
        
        if page_items == 0:
            search_loop = False
        else:
            for i in range(0, page_items):
                # 회사명
                job_company = soup.find_all('a', attrs={'class': 'name dev_view'})[i].text
                company_re = re.search('\(.*\)', job_company)
                if company_re:
                    str_filtering = job_company[company_re.span()[0]:company_re.span()[1]]
                    job_company = job_company.replace(str_filtering, '')
                job_company = job_company.replace('㈜', '')
                job_company = job_company.replace('㈔', '')
                job_company = job_company.replace('재)', '')
                job_company = job_company.replace('주)', '')
                job_company = job_company.replace(' ', '')
                job_company = job_company.replace('\n', '')
                if job_company in company_filtering_list:
                    continue
                    
                # 회사 링크
                company_link = base_link_korea + soup.find_all('a', attrs={'class': 'name dev_view'})[i]['href']
                
                # 채용공고
                job_posting = soup.find_all('a', attrs={'class': 'title dev_view'})[i].text
                job_posting = job_posting.replace('\r\n', '')
                job_posting = job_posting.strip()

                # 채용공고 링크
                posting_link = base_link_korea + soup.find_all('a', attrs={'class': 'title dev_view'})[i]['href']

                # Dictionary에 저장
                save_dictionary(job_company, company_link, job_posting, posting_link)
                
                item_count += 1
        current_page += 1
    print(f'{search_word}: {item_count}')
toc()

대기업
중소기업
대기업
국내비영리단체·협회·교육재단
국내비영리단체·협회·교육재단
국내비영리단체·협회·교육재단
이미지인식: 144
Elapsed time is 5.786570310592651 seconds.


In [7]:
len(job_group)

100

In [8]:
nCnt = 0
for i in job_group:
    for j in job_group[i]:
        nCnt += 1
print(nCnt)

144


### 사람인

In [ ]:
# 사람인 검색 옵션
# searchType=search
# searchword=컴퓨터비전         # 검색어
# loc_mcd=101000,102000         # 지역, 101000(서울),102000(경기)
# company_cd=0,1,2,3,4,5,6,7    # 채용제공구분 / 9(파견/대행), 10(헤드헌팅)
# search_optional_item=y
# search_done=y
# panel_count=y
# recruitPage=1                 # 현재 페이지뷰
# recruitSort=relation          # 정렬방법
# recruitPageCount=100          # 표시 개수
# inner_com_type=
# show_applied=                 # 지원한 공고 표시
# quick_apply=                  # 빠른 입사지원
# except_read=                  # 조회한 페이지 제외
# ai_head_hunting=
# mainSearch=n

In [4]:
# 사람인
# 저장된 키워드 별 검색

page_view_items = 100
company_cd='0,1,2,3,4,5,6,7'
tic()
for search_word in search_words:

    item_count = 0
    current_page = 1
    
    # 검색 결과 리스트 페이지 별 확인
    search_loop = True
    while(search_loop):
        search_link = f'https://www.saramin.co.kr/zf_user/search/recruit'\
            + f'?search_area=main'\
            + f'&search_done=y'\
            + f'&search_optional_item=n'\
            + f'&searchType=search'\
            + f'&recruitSort=relation'\
            + f'&searchword={search_word}'\
            + f'&recruitPage={current_page}'\
            + f'&recruitPageCount={page_view_items}'\
            + f'&company_cd={company_cd}'\
            + f'&mainSearch=y'

        response = requests.get(search_link, verify=False)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        if len(soup.find_all('div', attrs={'class': 'info_no_result'})) == 1:
            search_loop = False
        else:
            current_page += 1

            page_items = len(soup.find_all('h2', attrs={'class': 'job_tit'}))

            # 해당 페이지 리스트 (1 ~ 100)
            for i in range(1, page_items+1):
                elements = soup.select(f'div.content > div:nth-child({i})')[0]

                # 채용공고명
                job_posting = elements.find('a', attrs={'class': 'data_layer'})['title']

                # 회사명
                job_company = elements.find('a', attrs={'class': 'track_event data_layer'}).text
                
                company_re = re.search('\(.*\)', job_company)
                if company_re:
                    str_filtering = job_company[company_re.span()[0]:company_re.span()[1]]
                    job_company = job_company.replace(str_filtering, '')
                job_company = job_company.replace('㈜', '')
                job_company = job_company.replace('㈔', '')
                job_company = job_company.replace('재)', '')
                job_company = job_company.replace('주)', '')
                job_company = job_company.replace(' ', '')
                job_company = job_company.replace('\n', '')
                if job_company in company_filtering_list:
                    continue
                    
                # 채용공고 링크
                posting_link = base_link_saram + elements.find('a', attrs={'class': 'data_layer'})['href']

                # 회사 링크
                company_link = base_link_saram + elements.find('a', attrs={'class': 'track_event data_layer'})['href']

                # Dictionary에 저장
                save_dictionary(job_company, company_link, job_posting, posting_link)
                
                
                item_count += 1
                
    print(f'{search_word}: {item_count}')
toc()

이미지인식: 211
Elapsed time is 12.18005084991455 seconds.


In [5]:
len(job_group)

147

In [6]:
nCnt = 0
for i in job_group:
    for j in job_group[i]:
        nCnt += 1
print(nCnt)

211


In [37]:
# JSON 파일 쓰기
file_path = "config/job_group.json"
tf = open(file_path, 'w')
json.dump(job_group, tf)
tf.close()

In [2]:
# JSON 파일 불러오기
file_path = "config/job_group.json"
with open(file_path, 'r') as file:
    job_group = json.load(file)
print(len(job_group))

In [5]:
company_filtering_list = ["학원", "자산", "교육센터", "...", 
                          "중소기업(300명이하)",
                          "벤처기업",
                          "중견기업(300명이상)",
                          "대기업계열사·자회사",
                          "외국계기업",
                          "대기업",
                          "중소기업",
                          "매출수조원하는물류기업",
                          "전기전자대기업",
                          "머신러닝기반의비전기술개발전문기업",
                          "150명이상의스타트업",
                          "유망AI솔루션기업",
                          "초우량벤처기업",
                          "메타버스/AR서비스기업",
                          "영상진단솔루션전문기업",
                          "코스닥상장sw기업",
                          "코스닥상장사",
                          "[국내유망데이터솔류션개발업체]",
                          "국내대기업전자자동차전장사업부",
                          "국내비영리단체·협회·교육재단",
                          "현대CAD디자인직업전문학교"
                          ]